This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

# De Casteljau's Algorithm

In <cite data-cite="shoemake1985animating">Shoemake (1985)</cite>,
which famously introduces quaternions to the field of computer graphics,
Shoemake suggests to apply a variant of the
[De Casteljau's Algorithm](../euclidean/bezier-de-casteljau.ipynb)
to a quaternion control polygon,
using [Slerp](slerp.ipynb) instead of linear interpolations.

In [ ]:
def slerp(one, two, t):
    return (two * one.inverse())**t * one

In [ ]:
import numpy as np

[helper.py](helper.py)

In [ ]:
from helper import angles2quat, animate_rotations, display_animation

## "Cubic"

<cite data-cite="shoemake1985animating">Shoemake (1985)</cite>
only talks about the "cubic" case,
consisting of three nested applications of Slerp.

The resulting curve is of course not simply a polynomial of degree 3,
but something quite a bit more involved.
Therefore, we use the term "cubic" in quotes.

Shoemake doesn't talk about the "degree" of the curves at all,
they are only called "spherical Bézier curves".

In [ ]:
def de_casteljau(q0, q1, q2, q3, t):
    slerp_0_1 = slerp(q0, q1, t)
    slerp_1_2 = slerp(q1, q2, t)
    slerp_2_3 = slerp(q2, q3, t)
    return slerp(
        slerp(slerp_0_1, slerp_1_2, t),
        slerp(slerp_1_2, slerp_2_3, t),
        t,
    )

In [ ]:
q0 = angles2quat(45, 0, 0)
q1 = angles2quat(0, 0, 0)
q2 = angles2quat(-90, 90, -90)
q3 = angles2quat(-90, 0, 0)

In [ ]:
times = np.linspace(0, 1, 100)

In [ ]:
ani = animate_rotations(
    [de_casteljau(q0, q1, q2, q3, t) for t in times],
    figsize=(4, 3),
)

In [ ]:
display_animation(ani, default_mode='once')

## Arbitrary "Degree"

[splines.quaternion.DeCasteljau](../python-module/splines.quaternion.rst#splines.quaternion.DeCasteljau) class

In [ ]:
from splines.quaternion import DeCasteljau

In [ ]:
s = DeCasteljau([
    [
        angles2quat(0, 0, 0),
        angles2quat(90, 0, 0),
    ],
    [
        angles2quat(90, 0, 0),
        angles2quat(0, 0, 0),
        angles2quat(0, 90, 0),
    ],
    [
        angles2quat(0, 90, 0),
        angles2quat(0, 0, 0),
        angles2quat(-90, 0, 0),
        angles2quat(-90, 90, 0),
    ],
], grid=[0, 1, 3, 6])

In [ ]:
times = np.linspace(s.grid[0], s.grid[-1], 100)

In [ ]:
ani = animate_rotations(s.evaluate(times), figsize=(4, 3))

In [ ]:
display_animation(ani, default_mode='once')

## Constant Angular Speed

> Is there a way to construct a curve parameterized by arc length?
> This would be very useful.
>
> --<cite data-cite="shoemake1985animating">Shoemake (1985)</cite>, section 6: "Questions"

In [ ]:
from splines import ConstantSpeedAdapter

In [ ]:
s1 = DeCasteljau([[
    angles2quat(90, 0, 0),
    angles2quat(0, -45, 90),
    angles2quat(0, 0, 0),
    angles2quat(180, 0, 180),
]])

In [ ]:
s2 = ConstantSpeedAdapter(s1)

In [ ]:
ani = animate_rotations({
    'non-constant angular speed': s1.evaluate(np.linspace(s1.grid[0], s1.grid[-1], 100)),
    'constant angular speed': s2.evaluate(np.linspace(s2.grid[0], s2.grid[-1], 100)),
}, figsize=(6, 3))

In [ ]:
display_animation(ani, default_mode='reflect')

## Joining Curves

In section 4.2,
<cite data-cite="shoemake1985animating">Shoemake (1985)</cite>
provides two function definitions:

\begin{align*}
\operatorname{Double}(p, q) &= 2 (p \cdot q) q - p\\
\operatorname{Bisect}(p, q) &= \frac{p + q}{\|p + q\|}
\end{align*}

Given three successive key quaternions
$q_{n-1}$, $q_n$ and $q_{n+1}$,
these functions are used to compute
appropriate control quaternions
$b_n$ (controlling the incoming tangent to $q_n$) and
$a_n$ (controlling the outgoing tangent to $q_n$):

\begin{align*}
a_n &=
\operatorname{Bisect}(\operatorname{Double}(q_{n-1}, q_n), q_{n+1})\\
b_n &=
\operatorname{Double}(a_n, q_n)
\end{align*}

In [ ]:
def double(p, q):
    return 2 * p.dot(q) * q - p

In [ ]:
def bisect(p, q):
    return (p + q).normalize()

In [ ]:
def outgoing(q_1, q0, q1):
    return bisect(double(q_1, q0), q1)

Normalization is not explicitly mentioned in the paper,
but even though the results have a length very close to `1.0`,
we still have to call `normalize()` to turn the
[Quaternion](../python-module/splines.quaternion.rst#splines.quaternion.Quaternion)
result into a
[UnitQuaternion](../python-module/splines.quaternion.rst#splines.quaternion.UnitQuaternion).

In [ ]:
def incoming(a, q0):
    return double(a, q0).normalize()

In [ ]:
rotations = [
    angles2quat(0, 0, 0),
    angles2quat(45, 0, 0),
    angles2quat(90, 45, 0),
    angles2quat(90, 90, 0),
    angles2quat(91, 91, 0),
    angles2quat(180, 0, 90),
]

We don't want to worry about end conditions,
so we overlap beginning and end to get a closed curve:

In [ ]:
rotations += rotations[:2]

In [ ]:
control_points = []
for q_1, q0, q1 in zip(rotations, rotations[1:], rotations[2:]):
    a = outgoing(q_1, q0, q1)
    control_points.extend([incoming(a, q0), q0, q0, a])

In [ ]:
control_points = control_points[-2:] + control_points[:-2]

In [ ]:
segments = list(zip(*[iter(control_points)] * 4))

In [ ]:
from splines.quaternion import DeCasteljau

In [ ]:
s = DeCasteljau(segments)

In [ ]:
s.grid

In [ ]:
times = np.linspace(s.grid[0], s.grid[-1], 100)

In [ ]:
ani = animate_rotations(s.evaluate(times), figsize=(4, 3))

In [ ]:
display_animation(ani, default_mode='loop')